Install packages

In [ ]:
%pip install youtube-transcript-api google-generativeai chromadb

Import modules

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

# Gemini API
# Free, double-check pricing https://ai.google.dev/pricing
import google.generativeai as genai

# ChromaDB
import chromadb
from chromadb.utils import embedding_functions

import os

Set up resources

In [ ]:
# Load Gemini API key from .env file
# I have a 1-minute video on how to get your Gemini API key: https://youtu.be/Mx-OZOXvFi0
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# Instantiate Gemini model
# Model choices: https://ai.google.dev/gemini-api/docs/models/gemini
genai_model = genai.GenerativeModel('models/gemini-1.5-flash')

# Load the vector database, if it exists, otherwise create new on first run
chroma_client = chromadb.PersistentClient(path="my_vectordb")

# Select an embedding function.
# Embedding Function choices:https://docs.trychroma.com/guides/embeddings#custom-embedding-functions
gemini_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=GEMINI_API_KEY)

# Load collection, if it exists, otherwise create new on first run. Specify the model that we want to use to do the embedding.
chroma_collection = chroma_client.get_or_create_collection(name='yt_notes', embedding_function=gemini_ef)

INPUTS:

In [ ]:
# Some sample YouTube videos:
# https://youtu.be/IdLSZEYlWVo
# https://youtu.be/tL-wnMVyTQI
# https://youtu.be/etSdP9CFmko
# https://youtu.be/rgRIZDsEwCk
# https://youtu.be/_EA-74yr5D4

yt_video_id = 'hQH4-5o0BMM'

# Adjust prompt as needed
prompt = "Extract key notes from video transcript: "

Extract Transcript

In [ ]:
# Reference: https://github.com/jdepoix/youtube-transcript-api
transcript = YouTubeTranscriptApi.get_transcript(yt_video_id, languages=['en','en-US','en-GB'])
transcript = TextFormatter().format_transcript(transcript)

with open("temp_transcript.txt", "w") as file:
    file.write(transcript)

Generate Notes

In [ ]:
# https://ai.google.dev/api/generate-content
response = genai_model.generate_content(prompt + transcript, stream=False)

with open("temp_notes.txt", "w") as file:
    file.write(response.text)

# Review temp_notes.txt, edit if necessary

Save Notes

In [ ]:
with open("temp_notes.txt", "r") as file:
    notes = file.read()

# Insert, if record doesn't exist, otherwise update existing record
# https://docs.trychroma.com/reference/py-collection#upsert
chroma_collection.upsert(
    documents=[notes],
    ids=[yt_video_id]
)

# Validation
result = chroma_collection.get(yt_video_id, include=['documents'])
result

Search Notes

In [ ]:
query_text = "How much beef do I need for the beef ribs recipe?"
n_results = 5

# https://docs.trychroma.com/reference/py-collection#query
results = chroma_collection.query(
    query_texts=[query_text],
    n_results=n_results,
    include=['documents', 'distances', 'metadatas'],
)

for i in range(len(results['ids'][0])):
    id       = results["ids"][0][i]
    document = results['documents'][0][i]

    print("************************************************************************")
    print(f"{i+1}.  https://youtu.be/{id}")
    print("************************************************************************")
    print(document)


In [ ]:
prompt = "Answer the following QUESTION using DOCUMENT as context."
prompt += f"QUESTION: {query_text}"
prompt += f"DOCUMENT: {results['documents'][0][0]}"

response = genai_model.generate_content(prompt, stream=False)
print(response.text)